In [1]:
!pip install pulp

  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 186.2 kB/s eta 0:01:35
   ---------------------------------------- 0.0/17.7 MB 178.6 kB/s eta 0:01:39
   ---------------------------------------- 0.1/17.7 MB 261.4 kB/s eta 0:01:08
   ---------------------------------------- 0.1/17.7 MB 423.5 kB/s eta 0:00:42
   ---------------------------------------- 0.2/17.7 MB 588.9 kB/s eta 0:00:30
    --------------------------------------- 0.3/17.7 MB 682.7 

In [6]:
import pulp
import math

In [27]:
# Updated Drones Data with Quantity
drones = [
    {'name': 'Drone1', 'endurance': 5, 'coverage': 10, 'speed': 100, 'quantity': 2},  # 2 units of Drone1
    {'name': 'Drone2', 'endurance': 7, 'coverage': 15, 'speed': 120, 'quantity': 1},  # 1 unit of Drone2
     {'name': 'Drone3' ,'endurance': 10, 'coverage': 20, 'speed': 80, 'quantity': 2},
    {'name': 'Drone4' ,'endurance': 12, 'coverage': 15, 'speed': 75, 'quantity': 2},
    {'name': 'Drone5' ,'endurance': 8, 'coverage': 12, 'speed': 55, 'quantity': 2},
    {'name': 'Drone6' ,'endurance': 8, 'coverage': 18, 'speed': 60, 'quantity': 2},
]

# Points Data (same as before)
coordData = [
    {'lat': 1, 'lng': 1, 'order': 1, 'size': 100, 'time': 2},
    {'lat': 2, 'lng': 2, 'order': 2, 'size': 200, 'time': 3},
    {'lat': 3, 'lng': 3, 'order': 3, 'size': 300, 'time': 4},
]

# User input for minimum coverage percentage
min_coverage_percent = 90

In [28]:
# Haversine formula to calculate the great-circle distance between two points
def calculate_distance(point1, point2):
    R = 6371  # Earth radius in km
    lat1, lon1 = map(math.radians, [float(point1['lat']), float(point1['lng'])])
    lat2, lon2 = map(math.radians, [float(point2['lat']), float(point2['lng'])])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

# Update points_data with distances between consecutive points
for i in range(len(points_data) - 1):
    points_data[i]['distance_to_next'] = calculate_distance(points_data[i], points_data[i + 1])
points_data[-1]['distance_to_next'] = 0  # Last point has no next point



In [29]:
# Define the problem
problem = pulp.LpProblem("DroneDeploymentMultiplePoints", pulp.LpMaximize)

# Expanded decision variables for each drone instance
drone_vars = []
drone_indices = []  # Keep track of the drone index for each variable
for i, drone in enumerate(drones):
    for q in range(drone['quantity']):
        drone_vars.append(pulp.LpVariable(f"x_{i}_{q}", cat=pulp.LpBinary))
        drone_indices.append(i)  # Store the drone index

# Objective Function: Still maximizing coverage
problem += pulp.lpSum([var * drones[drone_indices[i]]['coverage'] for i, var in enumerate(drone_vars)]), "TotalCoverage"

# Constraints for each drone instance at each point
for idx, var in enumerate(drone_vars):
    drone_index = drone_indices[idx]
    drone = drones[drone_index]
    time_spent = 0
    for j, point in enumerate(points_data):
        if j < len(points_data) - 1:  # Exclude the last point which has no next point
            travel_time_to_next = point['distance_to_next'] / drone['speed']
            time_spent += travel_time_to_next + point['time']
            problem += var * time_spent <= drone['endurance'], f"Endurance_{idx}_Point_{j}"

# Constraint: Minimum coverage area requirement at each point
for j, point in enumerate(points_data):
    problem += pulp.lpSum([var * drones[drone_indices[i]]['coverage'] for i, var in enumerate(drone_vars)]) >= point['size'], f"MinCoverage_Point_{j}"

# Solve the problem
problem.solve()

# Output the results
print("Status:", pulp.LpStatus[problem.status])
for var in drone_vars:
    if pulp.value(var) == 1:
        print(f"{var.name} is selected.")


Status: Infeasible
x_0_0 is selected.
x_0_1 is selected.
x_1_0 is selected.
x_2_0 is selected.
x_2_1 is selected.
x_3_0 is selected.
x_3_1 is selected.
x_4_0 is selected.
x_4_1 is selected.
x_5_0 is selected.
x_5_1 is selected.
